## Example chat with pdf text as context


In [ ]:
# Set up to use local modules
%load_ext autoreload
%autoreload 2
import os


: 

In [ ]:
from pypdf import PdfReader
from src import utils

import pyprojroot

pdf_path = pyprojroot.here("data/vaswani_et_al_2017.pdf")

reader = PdfReader(pdf_path)  # creating a pdf reader object
print(len(reader.pages))  # number of pages in pdf file
page = reader.pages[0]  # getting a specific page from the pdf file
first_page_text = page.extract_text()  # extracting text from page
print(first_page_text)

In [ ]:
# extracting text from all pages

entire_text = utils.extract_text_from_pdf(pdf_path, max_n_pages=None)
print(entire_text)

In [ ]:
import textwrap
from IPython.display import Markdown


def to_markdown(text):
    text = text.replace("•", "  *")
    return Markdown(textwrap.indent(text, "> ", predicate=lambda _: True))

In [ ]:
# Use pdf text as context in Gemini chat, with engineered prompt to ask for summary
import google.generativeai as genai

model = genai.GenerativeModel("gemini-pro")

prompt_context_text = utils.extract_text_from_pdf(pdf_path, max_n_pages=2)

prompt = f"""
    Explain and summarize the scientifc results in the following text
    which has been extracted from a publication.
    First, provide a brief summary of the results in the results,
    then provide a bullet point list of concepts and results that a user may
    want to ask about.
    ---
    {prompt_context_text}
    ---
    """
prompt_n_tokens = model.count_tokens(prompt)
print(f"Prompt has {prompt_n_tokens} tokens")
response = model.generate_content(prompt)
to_markdown(response.text)

In [ ]:
response.candidates

## gemini-pro-vision for pdf analysis?


In [ ]:
model = genai.GenerativeModel("gemini-pro-vision")

prompt = """
You are a very professional document summarization specialist.
Please summarize the given document.
"""

from vertexai.generative_models import Part


pdf_file_uri = pdf_path
pdf_file = Part.from_uri(pdf_file_uri, mime_type="application/pdf")
contents = [pdf_file, prompt]

# prompt_n_tokens = model.count_tokens(prompt)
# print(f"Prompt has {prompt_n_tokens} tokens")
response = model.generate_content(contents)
to_markdown(response.text)

## Use preceding messages as context


In [ ]:
model = genai.GenerativeModel("gemini-pro")
chat = model.start_chat(history=[])
chat

In [ ]:
response = chat.send_message(
    "In one sentence, explain how a computer works to a young child."
)
to_markdown(response.text)

In [ ]:
chat.history

In [ ]:
for message in chat.history:
    display(to_markdown(f"**{message.role}**: {message.parts[0].text}"))

## Context, with generate_content


In [ ]:
model = genai.GenerativeModel("gemini-pro")
pre_prompt = """
    Instructions: Always answer like a pirate. Arr matey?
    """
messages = []
messages.append({"role": "user", "parts": [pre_prompt]})
messages.append({"role": "model", "parts": ["Arr matey!"]})
messages.append(
    {
        "role": "user",
        "parts": ["In one sentence, explain how a computer works to a young child."],
    },
)
response = model.generate_content(messages)
messages.append({"role": "model", "parts": [response.text]})
messages